# Notebook 4: Adding Image Post-Processing

Example: Acquire Process Decide
Acquisitions, Processes, and decisions cann be prebuilt from libraries and stiched 

The AcquisitonPLugin is a set of variables and callback functions that can be used to acquire a dataset.

The PostProcesingPipeline takes a dataset and returns ProcessedData. 

The Deicision takes ProcessedData and returns an acquisition.

This 3 step process enables advanced automation of image acquistion. 

In [ ]:
import sys,os
sys.path.append('source')

In [ ]:
from source.apd import *

## Example 4.1: Using the APD System to run an AcquisitionPlugin-PostProcessPipeline-Decision (APD)

In [ ]:
from source.apd import *
acqLibrary=AcquisitionPluginLibrary()
acquisition=acqLibrary.xySequence([[0,0]])

process=PostProcessPipeline()
process.add('mean')

decision=DecisionRepeatAcquisition()

apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') 
apdSystem.linkAPD(acquisition=acquisition,process=process,decision=DecisionRepeatAcquisition())
apdSystem.run()


## Example 4.X: Writing your own PostProcess to print Hello World

This section describes how to make your own PostProcessNode using a function callback.

The first section describes the callback schema for process node functions, and the secon section shows how to run the callbacks.


In [3]:
# First, we will create an aquisition from the last notebook.
import sys,os
sys.path.append('source')
from source.apd import *
from source.postprocessors import PostProcessPipeline as PostProcessPipeline
from source.postprocessors import PostProcessNode

from source.hooks import HookSet, Hook
from source.hooks import ImageHook

def imageProcessFunction(self,image,metadata,**kwargs):
    #print(self)
    #print(metadata)
    print(image.shape)
    image = image[:250,:]
    metadata['Height'], metadata['Width'] = image.shape
    return image, metadata

hooks=HookSet()
hooks.hookImageProcess=ImageHook(function=imageProcessFunction)

apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') #local
lib = AcquisitionPluginLibrary()
positions=[[0,0],[1,0],[0,1]] 
acquisition = lib.xySequence(positions)
acquisition.hooks=hooks
acquisition.settings.show_display=False

In [5]:
# Section 1: making the Hello and World callback functions

def hello(self,chunks,acq,*args,**kwargs):
    output={}
    print('Hello ')
    print([chunk.shape for chunk in chunks])
    return (chunks,output)

def world(self,chunks,acq,*args,**kwargs):
    output={}
    print('World ')
    return (chunks, output)

# Putting the functions into the pipeline and running it

process=PostProcessPipeline()
process.addNode(PostProcessNode(function=hello))
#process.addNode(PostProcessNode(function=world))

decision=DecisionRepeatAcquisition()

#apdSystem.linkAPD(acquisition=acquisition,process=processors,decision=DecisionRepeatAcquisition())
apdSystem = APDSystem(configFileName='myConfig.cfg', rootDataFolder='') #local
apdSystem.linkAPD(acquisition=acquisition,process=process,decision=decision)
apdSystem.run()

#dataset=apdSystem.acquire(acquisition)

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

#data=NDTiffDataset('data/core/xyLooseGrid_1') # Load Sample dataset
#acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing




(512, 512)
(512, 512)
(512, 512)
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}]}
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}], frozenset({('position', 1), ('time', 0)}): [{'position': 1, 'time': 0}]}
{frozenset({('position', 0), ('time', 0)}): [{'position': 0, 'time': 0}], frozenset({('position', 1), ('time', 0)}): [{'position': 1, 'time': 0}], frozenset({('position', 2), ('time', 0)}): [{'time': 0, 'position': 2}]}
Hello 


AttributeError: 'list' object has no attribute 'shape'

In [ ]:
def cropImage(self,chunks,acq,*args,**kwargs):
    output={}
    for i in range(len(chunks)):
        chunks[i]=chunks[i][10:,:-10] 
    return (chunks,output)

def averageImageValue(self,chunks,acq,*args,**kwargs):
    output={}
    for i in range(len(chunks)):
        print(chunks[i].shape)
        output['averageImageValue']=np.mean(chunks[i])
    return (chunks,output)


# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=cropImage))
processors.addNode(PostProcessNode(function=averageImageValue))

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_1') # Load Sample dataset
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing
processors.process(data,acq)

## Example X.X: Using Cell Detection on Two Color Images

In [ ]:
def detectCells(self,chunks,acq,*args,**kwargs):
    output={}
    detector=CellDetectorCellMask()
    for i in range(len(chunks)):
        print(chunks[i].shape)
        output['averageImageValue']=detector.process(chunks[i])
    return (chunks,output)

# Putting the functions into the pipeline and running it

processors=PostProcessPipeline()
processors.addNode(PostProcessNode(function=detectCells,squish_axes='channel')) # squish_axes combines images long the channel dimension

# Running the function. <data> and <acq> are just arbitrary things needed to get the callback to work in this particular scenario.

data=NDTiffDataset('data/core/xyLooseGrid_5') # Load Sample dataset thats is 2 channel
acq=AcquisitionPlugin() #make an empty acquisition to pass into for testing
processedData=processors.process(data,acq)

ADD DISTRIBUTED COMPUTING!!!!!

TRANSFRER runners to BRIANS 